In [102]:
### import radd
from radd import build
from radd.tools import analyze
from itertools import takewhile
from scipy.stats.mstats_extras import mjci
homedir = os.path.expanduser('~')

ssb = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SSBayes/alldata_clean.csv"))
udf = ssb[ssb.cond.isin(['uniform'])]
edf = ssb[ssb.cond.isin(['early'])]
ldf = ssb[ssb.cond.isin(['late'])]

re = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/SS/Reactive/Re_Data.csv"), index_col=0)
cox = pd.read_csv(os.path.join(homedir, "Dropbox/Projects/Coxon/coxon_behav_data_binned.csv"))

inits = {'a': 0.45, 'v': 1.26, 'xb': 1.3, 'ssv': -1.3, 'tr': 0.2}

In [126]:
m = build.Model(data=redata, inits=inits, kind='xdpm', fit_on='average')

In [127]:
%%timeit 
m.opt.simulator.sim_fx(p, analyze=True)

10 loops, best of 3: 119 ms per loop


In [125]:
%%timeit
p = sim.vectorize_params(inits)
dvg_input = sim.__update_go_process__(p)
dvg = sim.go_process(dvg_input)
Ps, Ts = sim.__update_stop_process__(p)
ss_onsets = np.where(Ts<dvg_input[1][:, None], dvg_input[1][:, None]-Ts, 0)
sinits = sim.init_ss((dvg[:, :sim.nss, :].reshape(sim.nlevels, sim.nssd, sim.nss_per_ssd, dvg.shape[-1]), ss_onsets))
dvs = sim.stop_process((sinits, Ps, Ts))
sim.analyze_fx(dvg, dvs, p)

10 loops, best of 3: 117 ms per loop


In [96]:
ss_onsets

array([[  0,  50, 100, 150, 200]])

In [114]:
sim=m.opt.simulator



In [66]:
import numba

def simulate_dpm(sim, p, analyze=True):
    """ Simulate the dependent process model (DPM)
    ::Arguments::
        p (dict):
            parameter dictionary. values can be single floats
            or vectors where each element is the value of that
            parameter for a given condition
        analyze (bool <True>):
            if True (default) return rt and accuracy information
            (yhat in cost fx). If False, return Go and Stop proc.
    ::Returns::
        yhat of cost vector (ndarray)
        or Go & Stop processes in list (list of ndarrays)
    """

    p = sim.vectorize_params(p)
    Pg, Tg = sim.__update_go_process__(p)
    Ps, Ts = sim.__update_stop_process__(p)
    ss_onsets = np.where(Ts<Tg[:, None], Tg[:, None]-Ts, 0)

    ntot = sim.ntot
    nl = sim.nlevels
    nss = sim.nss
    nssd = sim.nssd
    ssd_ix = np.arange(nssd)
    nss_per = sim.nss_per_ssd
    dx = sim.dx

    DVg = sim.xtb[:, None] * csum(np.where((rs((nl, ntot, Tg.max())).T < Pg), dx, -dx).T, axis=2)
    dvg_ss = DVg[:, :nss, :].reshape(nl, nssd, nss_per, DVg.shape[-1])
    # initialize stop-process (DVs) FROM value of go-process (DVg) at t=SSD
    init_ss = np.array([dvg_ss[i, ssd_ix, :, ix] for i, ix in enumerate(ss_onsets)])
    DVs = csum(np.where(rs((nl, nssd, nss_per, Ts.max())) < Ps, dx, -dx), axis=3)
    if analyze:
        return sim.analyze_fx(DVg, DVs, p)
#     else:
#         return [DVg, DVs]

In [67]:
%timeit simulate_dpm(sim, m.inits)

10 loops, best of 3: 115 ms per loop


In [31]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [25]:
#!/usr/local/bin/env python
from __future__ import division
from copy import deepcopy
import numpy as np
from numpy import array
from numpy.random import sample as rs
from numpy import hstack as hs
from numpy import newaxis as na
from numpy import cumsum as csum
from scipy.stats.mstats import mquantiles as mq

In [38]:
%timeit simulate_dpm(m.inits)

TypeError: simulate_dpm() takes at least 2 positional arguments (1 given)

In [48]:
simulate_dpm(sim, m.inits)

In [13]:

%timeit m.opt.simulator.sim_fx(m.inits)

10 loops, best of 3: 118 ms per loop
